In [1]:

# 라이브러리 임포트 
# sklearn, librosa, keras, pickle

from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

from random import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import librosa.display
from matplotlib.pyplot import specgram
import os
import IPython.display as ipd
import random
import pickle
import joblib

import wave
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import rmsprop


Using TensorFlow backend.


In [2]:
'''
함수 구성

1. 데이터 시각화
2. 데이터 분할 및 셔플
3. 정확도 계산


'''

#list를 판다스 df로 보여주기
def print_df(mylist):
    df = pd.DataFrame(data = mylist)
    print(df)


# 데이터와 라벨 연결 후 셔플
def train_test_split(x, y, train_size):
    
    # 두 리스트를 합쳐서, 셔플하여 조정
    # 이후 train size 만큼 나눠서 분배
    
    tmp = [[a,b] for a, b in zip(x, y)]
    random.shuffle(tmp)
    x2 = [n[0] for n in tmp]
    y2 = [n[1] for n in tmp]
    index = int(len(y) * train_size)
    x_train = []
    x_test = []
    x_train = x2[:index]
    x_test = x2[index:]
    y_train = y2[:index]
    y_test = y2[index:]


    return x_train, x_test, y_train, y_test

# 정확도 측정
def accuracy_score(error,total):
    return (total - error) / total





In [3]:
# 학습 모델
# Based on https://www.kaggle.com/dasanurag38/audio-emotion-recognition
# 그러나 위 링크는 데이터 중복 사용, 그 부분을 반영하여 모델 일부 수정함.
# SVM 은 sklearn 을 이용하여 바로 처리
# 모델 개선 여지 많음

def create_model_LSTM():
    # LSTM model
    
    model = Sequential()
    model.add(LSTM(256, return_sequences=False, input_shape=(40, 1)))
    model.add(Dense(64))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(8))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def create_model_CNN_Final():
    # CNN model_Fianl(CNN)
    model = Sequential()
    model.add(Conv1D(64, kernel_size = 4, input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(64,kernel_size = 4))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Conv1D(128, kernel_size = 4))
    model.add(Activation('relu'))
    model.add(Conv1D(128, kernel_size = 4))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.25))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

def create_model_CNN_D():
    # CNN_D model
    model = Sequential()
    model.add(Conv1D(8, 5,padding='same', input_shape=(40, 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(16, 5,padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(32, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(16, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = rmsprop(lr=0.00001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])
    return model

def create_model_CNN_O():
    # CNN_O model
    model = Sequential()
    model.add(Conv1D(128, 5,padding='same',
                 input_shape=(40,1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(128, 5,padding='same',))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)
    

    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    return model




In [4]:
# 데이터 입력 및 MFCC 변환
# 정확도 향상을 위하여 남성, 여성 데이터 분리
# 추후 개인별 정규화 과정 필요 예상

root_dir = "./ravdess-video/" 

# 데이터 및 라벨 별도 정리

male_data = [] 
male_labels = [] 
female_data = [] 
female_labels = []

# 파일 이름을 통해 분류하여 리스트에 저장

for subdirs, dirs, files in os.walk(root_dir):
    for file in files:
        
        # librosa 활용
        y, sr = librosa.load(os.path.join(subdirs,file))
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
        if int(file[18:20])%2 != 0:
            male_data.append(mfccs)
            male_labels.append(int(file[7:8]) - 1)
        else:
            female_data.append(mfccs)
            female_labels.append(int(file[7:8]) - 1)
            
            
        

In [7]:

# np array 처리
m_data = np.array(male_data)
m_labels= np.array(male_labels)
f_data = np.array(female_data)
f_labels= np.array(female_labels)


# MFCC 변환에 시간이 오래 걸리므로, 별도의 파일로 저장
save_md = joblib.dump(m_data,'md.joblib')
save_fd = joblib.dump(m_labels,'ml.joblib')
save_ml = joblib.dump(f_data,'fd.joblib')
save_fl = joblib.dump(f_labels,'fl.joblib')


In [4]:
# 파일 로드
m_data = joblib.load('md.joblib')
m_labels= joblib.load('ml.joblib')
f_data = joblib.load('fd.joblib')
f_labels= joblib.load('fl.joblib')

In [5]:
# 88%의 데이터를 학습에 사용. 나머지를 테스트 데이터로 설정

m_train, m_test, ml_train, ml_test = train_test_split(m_data, m_labels, 0.88)
f_train, f_test, fl_train, fl_test = train_test_split(f_data, f_labels, 0.88)


In [6]:
# 라벨 카테고리화

md_train = np.array(m_train)
md_test = np.array(m_test)
fd_train = np.array(f_train)
fd_test = np.array(f_test)

m_c1 =  to_categorical(ml_train)
m_c2 =  to_categorical(ml_test)
f_c1 = to_categorical(fl_train)
f_c2 = to_categorical(fl_test)

print(md_test.shape)


(150, 40)


In [161]:
# LSTM male 학습, 이름은 순서대로 ABCD로 명명
model_A = create_model_LSTM()
ns = int(md_test.shape[0]*0.5)
history = model_A.fit(np.expand_dims(md_train,-1),  m_c1, validation_data=(np.expand_dims(md_test[:ns], -1), m_c2[:ns]), epochs=200)

Train on 1098 samples, validate on 75 samples
Epoch 1/200
1098/1098 [==============================] - 14s 13ms/step - loss: 2.0600 - acc: 0.1594 - val_loss: 2.0367 - val_acc: 0.1867
Epoch 2/200
1098/1098 [==============================] - 5s 5ms/step - loss: 2.0166 - acc: 0.1995 - val_loss: 1.9957 - val_acc: 0.1600
Epoch 3/200
1098/1098 [==============================] - 5s 5ms/step - loss: 1.9634 - acc: 0.2250 - val_loss: 1.9394 - val_acc: 0.2133
Epoch 4/200
1098/1098 [==============================] - 6s 5ms/step - loss: 1.9086 - acc: 0.2341 - val_loss: 1.8322 - val_acc: 0.2533
Epoch 5/200
1098/1098 [==============================] - 5s 5ms/step - loss: 1.8481 - acc: 0.2541 - val_loss: 1.7937 - val_acc: 0.2400
Epoch 6/200
1098/1098 [==============================] - 6s 5ms/step - loss: 1.7953 - acc: 0.2887 - val_loss: 1.6936 - val_acc: 0.2800
Epoch 7/200
1098/1098 [==============================] - 5s 5ms/step - loss: 1.7719 - acc: 0.2705 - val_loss: 1.6198 - val_acc: 0.2667
Epoch 8

Epoch 61/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.7503 - acc: 0.7368 - val_loss: 1.0111 - val_acc: 0.5600
Epoch 62/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.7477 - acc: 0.7140 - val_loss: 1.0765 - val_acc: 0.5867
Epoch 63/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.7309 - acc: 0.7222 - val_loss: 1.1062 - val_acc: 0.5867
Epoch 64/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.7255 - acc: 0.7240 - val_loss: 1.1279 - val_acc: 0.6000
Epoch 65/200
1098/1098 [==============================] - 6s 5ms/step - loss: 0.7115 - acc: 0.7404 - val_loss: 1.1522 - val_acc: 0.6133
Epoch 66/200
1098/1098 [==============================] - 6s 5ms/step - loss: 0.6385 - acc: 0.7587 - val_loss: 1.2476 - val_acc: 0.5867
Epoch 67/200
1098/1098 [==============================] - 7s 6ms/step - loss: 0.6799 - acc: 0.7523 - val_loss: 1.3384 - val_acc: 0.5333
Epoch 68/200
1098/1098 [========================

1098/1098 [==============================] - 5s 5ms/step - loss: 0.4092 - acc: 0.8834 - val_loss: 1.7549 - val_acc: 0.6267
Epoch 122/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.3290 - acc: 0.8898 - val_loss: 2.0358 - val_acc: 0.6267
Epoch 123/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.2621 - acc: 0.9126 - val_loss: 1.8926 - val_acc: 0.6000
Epoch 124/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.2069 - acc: 0.9353 - val_loss: 2.3987 - val_acc: 0.5733
Epoch 125/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.1972 - acc: 0.9344 - val_loss: 1.9196 - val_acc: 0.6533
Epoch 126/200
1098/1098 [==============================] - 5s 5ms/step - loss: 0.1274 - acc: 0.9581 - val_loss: 2.2748 - val_acc: 0.6400
Epoch 127/200
1098/1098 [==============================] - 6s 5ms/step - loss: 0.1599 - acc: 0.9490 - val_loss: 2.2902 - val_acc: 0.6133
Epoch 128/200
1098/1098 [==============================

Epoch 181/200
1098/1098 [==============================] - 6s 5ms/step - loss: 0.2243 - acc: 0.9472 - val_loss: 2.4259 - val_acc: 0.6133
Epoch 182/200
1098/1098 [==============================] - 6s 5ms/step - loss: 0.1118 - acc: 0.9645 - val_loss: 2.3332 - val_acc: 0.5867
Epoch 183/200
1098/1098 [==============================] - 6s 5ms/step - loss: 0.0896 - acc: 0.9718 - val_loss: 2.6361 - val_acc: 0.5733
Epoch 184/200
1098/1098 [==============================] - 6s 6ms/step - loss: 0.0954 - acc: 0.9791 - val_loss: 3.1313 - val_acc: 0.5600
Epoch 185/200
1098/1098 [==============================] - 8s 7ms/step - loss: 0.1435 - acc: 0.9545 - val_loss: 3.3090 - val_acc: 0.5733
Epoch 186/200
1098/1098 [==============================] - 7s 6ms/step - loss: 0.0545 - acc: 0.9854 - val_loss: 2.9535 - val_acc: 0.5733
Epoch 187/200
1098/1098 [==============================] - 6s 6ms/step - loss: 0.1486 - acc: 0.9627 - val_loss: 2.6519 - val_acc: 0.5867
Epoch 188/200
1098/1098 [================

In [154]:
# LSTM female 학습
model_A2 = create_model_LSTM()
ns2 = int(fd_test.shape[0]*0.5)
history = model_A2.fit(np.expand_dims(fd_train,-1),  f_c1, validation_data=(np.expand_dims(fd_test[:ns2], -1), f_c2[:ns2]), epochs=200)

Train on 1059 samples, validate on 72 samples
Epoch 1/200
1059/1059 [==============================] - 13s 13ms/step - loss: 2.0471 - acc: 0.1624 - val_loss: 2.0360 - val_acc: 0.1667
Epoch 2/200
1059/1059 [==============================] - 4s 4ms/step - loss: 2.0079 - acc: 0.1690 - val_loss: 1.9602 - val_acc: 0.2500
Epoch 3/200
1059/1059 [==============================] - 4s 4ms/step - loss: 1.9739 - acc: 0.1870 - val_loss: 1.9320 - val_acc: 0.2222
Epoch 4/200
1059/1059 [==============================] - 4s 4ms/step - loss: 1.9526 - acc: 0.2096 - val_loss: 1.8429 - val_acc: 0.3333
Epoch 5/200
1059/1059 [==============================] - 4s 4ms/step - loss: 1.8626 - acc: 0.2483 - val_loss: 1.8377 - val_acc: 0.2917
Epoch 6/200
1059/1059 [==============================] - 4s 4ms/step - loss: 1.8714 - acc: 0.2380 - val_loss: 1.8813 - val_acc: 0.3333
Epoch 7/200
1059/1059 [==============================] - 4s 4ms/step - loss: 1.8496 - acc: 0.2408 - val_loss: 1.7135 - val_acc: 0.3194
Epoch 8

Epoch 61/200
1059/1059 [==============================] - 4s 4ms/step - loss: 0.9711 - acc: 0.6289 - val_loss: 1.0632 - val_acc: 0.5556
Epoch 62/200
1059/1059 [==============================] - 4s 4ms/step - loss: 0.9991 - acc: 0.6195 - val_loss: 1.1830 - val_acc: 0.5694
Epoch 63/200
1059/1059 [==============================] - 4s 4ms/step - loss: 0.9419 - acc: 0.6364 - val_loss: 1.0314 - val_acc: 0.5417
Epoch 64/200
1059/1059 [==============================] - 4s 4ms/step - loss: 0.9115 - acc: 0.6553 - val_loss: 1.1772 - val_acc: 0.5833
Epoch 65/200
1059/1059 [==============================] - 4s 4ms/step - loss: 0.8893 - acc: 0.6308 - val_loss: 1.1468 - val_acc: 0.5139
Epoch 66/200
1059/1059 [==============================] - 4s 4ms/step - loss: 0.8661 - acc: 0.6771 - val_loss: 1.2073 - val_acc: 0.6111
Epoch 67/200
1059/1059 [==============================] - 4s 4ms/step - loss: 0.8002 - acc: 0.6874 - val_loss: 1.4638 - val_acc: 0.5417
Epoch 68/200
1059/1059 [========================

1059/1059 [==============================] - 5s 4ms/step - loss: 0.4740 - acc: 0.8300 - val_loss: 1.7020 - val_acc: 0.5972
Epoch 122/200
1059/1059 [==============================] - 5s 5ms/step - loss: 0.6363 - acc: 0.8045 - val_loss: 1.5445 - val_acc: 0.6111
Epoch 123/200
1059/1059 [==============================] - 5s 4ms/step - loss: 0.4641 - acc: 0.8432 - val_loss: 1.4345 - val_acc: 0.6528
Epoch 124/200
1059/1059 [==============================] - 5s 5ms/step - loss: 0.3786 - acc: 0.8782 - val_loss: 1.8390 - val_acc: 0.6111
Epoch 125/200
1059/1059 [==============================] - 5s 4ms/step - loss: 0.2617 - acc: 0.9122 - val_loss: 1.6639 - val_acc: 0.6528
Epoch 126/200
1059/1059 [==============================] - 5s 4ms/step - loss: 0.2401 - acc: 0.9169 - val_loss: 1.8921 - val_acc: 0.6806
Epoch 127/200
1059/1059 [==============================] - 5s 4ms/step - loss: 0.2236 - acc: 0.9292 - val_loss: 1.9924 - val_acc: 0.6667
Epoch 128/200
1059/1059 [==============================

Epoch 181/200
1059/1059 [==============================] - 5s 5ms/step - loss: 0.4989 - acc: 0.8791 - val_loss: 1.9398 - val_acc: 0.6667
Epoch 182/200
1059/1059 [==============================] - 5s 5ms/step - loss: 0.3149 - acc: 0.9169 - val_loss: 2.2965 - val_acc: 0.5556
Epoch 183/200
1059/1059 [==============================] - 5s 5ms/step - loss: 0.2437 - acc: 0.9273 - val_loss: 2.4257 - val_acc: 0.5833
Epoch 184/200
1059/1059 [==============================] - 5s 5ms/step - loss: 0.1287 - acc: 0.9594 - val_loss: 2.4273 - val_acc: 0.6250
Epoch 185/200
1059/1059 [==============================] - 5s 4ms/step - loss: 0.1080 - acc: 0.9726 - val_loss: 2.6128 - val_acc: 0.6111
Epoch 186/200
1059/1059 [==============================] - 5s 5ms/step - loss: 0.0905 - acc: 0.9688 - val_loss: 3.0391 - val_acc: 0.6111
Epoch 187/200
1059/1059 [==============================] - 5s 4ms/step - loss: 0.1070 - acc: 0.9726 - val_loss: 2.7515 - val_acc: 0.6250
Epoch 188/200
1059/1059 [================

In [7]:
# CNN Final male 학습
model_B = create_model_CNN_Final()
ns = int(md_test.shape[0]*0.5)
history = model_B.fit(np.expand_dims(md_train,-1),  m_c1, validation_data=(np.expand_dims(md_test[:ns], -1), m_c2[:ns]), epochs=60)

W1023 12:13:05.576321  3824 deprecation_wrapper.py:119] From C:\Users\JW\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1023 12:13:05.595270  3824 deprecation_wrapper.py:119] From C:\Users\JW\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1023 12:13:05.597264  3824 deprecation_wrapper.py:119] From C:\Users\JW\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1023 12:13:05.633172  3824 deprecation_wrapper.py:119] From C:\Users\JW\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1023 12:13:05.671034  3824 deprecation_wrapper.py:119] From C:\Users\JW\Anaconda3\lib\site

Train on 1098 samples, validate on 75 samples
Epoch 1/60
1098/1098 [==============================] - 1s 902us/step - loss: 2.0024 - acc: 0.1985 - val_loss: 1.9105 - val_acc: 0.2667
Epoch 2/60
1098/1098 [==============================] - 0s 204us/step - loss: 1.8620 - acc: 0.2659 - val_loss: 1.8399 - val_acc: 0.2133
Epoch 3/60
1098/1098 [==============================] - 0s 200us/step - loss: 1.7889 - acc: 0.2896 - val_loss: 1.7843 - val_acc: 0.3067
Epoch 4/60
1098/1098 [==============================] - 0s 204us/step - loss: 1.6919 - acc: 0.3452 - val_loss: 1.6138 - val_acc: 0.3200
Epoch 5/60
1098/1098 [==============================] - 0s 202us/step - loss: 1.5541 - acc: 0.4180 - val_loss: 1.5900 - val_acc: 0.4267
Epoch 6/60
1098/1098 [==============================] - 0s 205us/step - loss: 1.4479 - acc: 0.4754 - val_loss: 1.5687 - val_acc: 0.4267
Epoch 7/60
1098/1098 [==============================] - 0s 203us/step - loss: 1.3898 - acc: 0.4745 - val_loss: 1.4571 - val_acc: 0.4267
Ep

In [151]:
# CNN Final female 학습
model_B2 = create_model_CNN_Final()
ns2 = int(fd_test.shape[0]*0.5)
history = model_B2.fit(np.expand_dims(fd_train,-1),  f_c1, validation_data=(np.expand_dims(fd_test[:ns2], -1), f_c2[:ns2]), epochs=60)

Train on 1059 samples, validate on 72 samples
Epoch 1/60
1059/1059 [==============================] - 6s 5ms/step - loss: 2.0216 - acc: 0.1907 - val_loss: 1.9091 - val_acc: 0.1528
Epoch 2/60
1059/1059 [==============================] - 1s 682us/step - loss: 1.8492 - acc: 0.2446 - val_loss: 1.7575 - val_acc: 0.2361
Epoch 3/60
1059/1059 [==============================] - 1s 705us/step - loss: 1.6837 - acc: 0.3258 - val_loss: 1.6216 - val_acc: 0.3194
Epoch 4/60
1059/1059 [==============================] - 1s 689us/step - loss: 1.5192 - acc: 0.4117 - val_loss: 1.5051 - val_acc: 0.4306
Epoch 5/60
1059/1059 [==============================] - 1s 687us/step - loss: 1.4550 - acc: 0.4693 - val_loss: 1.5128 - val_acc: 0.3611
Epoch 6/60
1059/1059 [==============================] - 1s 705us/step - loss: 1.3036 - acc: 0.5080 - val_loss: 1.4767 - val_acc: 0.3750
Epoch 7/60
1059/1059 [==============================] - 1s 689us/step - loss: 1.2158 - acc: 0.5496 - val_loss: 1.4380 - val_acc: 0.4722
Epoc

In [144]:
# CNN D male 학습
model_C = create_model_CNN_D()
history = model_C.fit(np.expand_dims(md_train,-1),  m_c1, validation_data=(np.expand_dims(md_test[:ns], -1), m_c2[:ns]), epochs=700)

Train on 1098 samples, validate on 75 samples
Epoch 1/700
1098/1098 [==============================] - 5s 5ms/step - loss: 7.1883 - acc: 0.1357 - val_loss: 2.6940 - val_acc: 0.0933
Epoch 2/700
1098/1098 [==============================] - 0s 337us/step - loss: 2.8198 - acc: 0.1639 - val_loss: 2.1607 - val_acc: 0.1600
Epoch 3/700
1098/1098 [==============================] - 0s 321us/step - loss: 2.1617 - acc: 0.1794 - val_loss: 2.0667 - val_acc: 0.1467
Epoch 4/700
1098/1098 [==============================] - 0s 335us/step - loss: 2.0449 - acc: 0.2013 - val_loss: 2.0122 - val_acc: 0.1867
Epoch 5/700
1098/1098 [==============================] - 0s 326us/step - loss: 1.9689 - acc: 0.2459 - val_loss: 1.9259 - val_acc: 0.2133
Epoch 6/700
1098/1098 [==============================] - 0s 324us/step - loss: 1.9385 - acc: 0.2386 - val_loss: 1.9077 - val_acc: 0.2267
Epoch 7/700
1098/1098 [==============================] - 0s 354us/step - loss: 1.8738 - acc: 0.2450 - val_loss: 1.8150 - val_acc: 0.30

1098/1098 [==============================] - 0s 311us/step - loss: 0.6952 - acc: 0.7286 - val_loss: 0.9828 - val_acc: 0.6400
Epoch 120/700
1098/1098 [==============================] - 0s 304us/step - loss: 0.6586 - acc: 0.7477 - val_loss: 0.9451 - val_acc: 0.6400
Epoch 121/700
1098/1098 [==============================] - 0s 307us/step - loss: 0.6645 - acc: 0.7541 - val_loss: 0.9327 - val_acc: 0.6800
Epoch 122/700
1098/1098 [==============================] - 0s 305us/step - loss: 0.6050 - acc: 0.7769 - val_loss: 0.9028 - val_acc: 0.6533
Epoch 123/700
1098/1098 [==============================] - 0s 312us/step - loss: 0.6366 - acc: 0.7614 - val_loss: 1.0562 - val_acc: 0.6533
Epoch 124/700
1098/1098 [==============================] - 0s 303us/step - loss: 0.6221 - acc: 0.7687 - val_loss: 1.0426 - val_acc: 0.6533
Epoch 125/700
1098/1098 [==============================] - 0s 305us/step - loss: 0.6297 - acc: 0.7632 - val_loss: 0.9856 - val_acc: 0.6667
Epoch 126/700
1098/1098 [================

1098/1098 [==============================] - 0s 309us/step - loss: 0.4437 - acc: 0.8315 - val_loss: 1.0725 - val_acc: 0.6533
Epoch 179/700
1098/1098 [==============================] - 0s 303us/step - loss: 0.4939 - acc: 0.8179 - val_loss: 1.1040 - val_acc: 0.6267
Epoch 180/700
1098/1098 [==============================] - 0s 304us/step - loss: 0.4780 - acc: 0.8206 - val_loss: 1.2237 - val_acc: 0.6533
Epoch 181/700
1098/1098 [==============================] - 0s 304us/step - loss: 0.4582 - acc: 0.8388 - val_loss: 1.2209 - val_acc: 0.6133
Epoch 182/700
1098/1098 [==============================] - 0s 302us/step - loss: 0.4805 - acc: 0.8133 - val_loss: 1.0717 - val_acc: 0.6267
Epoch 183/700
1098/1098 [==============================] - 0s 310us/step - loss: 0.4625 - acc: 0.8379 - val_loss: 1.1174 - val_acc: 0.6133
Epoch 184/700
1098/1098 [==============================] - 0s 305us/step - loss: 0.4547 - acc: 0.8388 - val_loss: 1.1416 - val_acc: 0.6667
Epoch 185/700
1098/1098 [================

1098/1098 [==============================] - 0s 312us/step - loss: 0.3761 - acc: 0.8561 - val_loss: 1.1549 - val_acc: 0.6000
Epoch 238/700
1098/1098 [==============================] - 0s 307us/step - loss: 0.3705 - acc: 0.8616 - val_loss: 1.2625 - val_acc: 0.6400
Epoch 239/700
1098/1098 [==============================] - 0s 306us/step - loss: 0.3926 - acc: 0.8388 - val_loss: 1.1924 - val_acc: 0.6400
Epoch 240/700
1098/1098 [==============================] - 0s 312us/step - loss: 0.3838 - acc: 0.8552 - val_loss: 1.3092 - val_acc: 0.6400
Epoch 241/700
1098/1098 [==============================] - 0s 302us/step - loss: 0.3991 - acc: 0.8497 - val_loss: 1.1778 - val_acc: 0.6400
Epoch 242/700
1098/1098 [==============================] - 0s 305us/step - loss: 0.3778 - acc: 0.8607 - val_loss: 1.2549 - val_acc: 0.6133
Epoch 243/700
1098/1098 [==============================] - 0s 310us/step - loss: 0.3495 - acc: 0.8798 - val_loss: 1.4038 - val_acc: 0.6133
Epoch 244/700
1098/1098 [================

1098/1098 [==============================] - 0s 303us/step - loss: 0.3155 - acc: 0.8770 - val_loss: 1.3803 - val_acc: 0.6133
Epoch 297/700
1098/1098 [==============================] - 0s 308us/step - loss: 0.3187 - acc: 0.8871 - val_loss: 1.4254 - val_acc: 0.6400
Epoch 298/700
1098/1098 [==============================] - 0s 303us/step - loss: 0.3072 - acc: 0.8980 - val_loss: 1.3949 - val_acc: 0.6000
Epoch 299/700
1098/1098 [==============================] - 0s 313us/step - loss: 0.2788 - acc: 0.8989 - val_loss: 1.3630 - val_acc: 0.6400
Epoch 300/700
1098/1098 [==============================] - 0s 309us/step - loss: 0.2780 - acc: 0.8980 - val_loss: 1.3993 - val_acc: 0.6400
Epoch 301/700
1098/1098 [==============================] - 0s 306us/step - loss: 0.2810 - acc: 0.8989 - val_loss: 1.2714 - val_acc: 0.6267
Epoch 302/700
1098/1098 [==============================] - 0s 307us/step - loss: 0.2911 - acc: 0.9053 - val_loss: 1.2433 - val_acc: 0.6267
Epoch 303/700
1098/1098 [================

...

1098/1098 [==============================] - 0s 323us/step - loss: 0.2389 - acc: 0.9180 - val_loss: 1.3825 - val_acc: 0.6267
Epoch 415/700
1098/1098 [==============================] - 0s 306us/step - loss: 0.2365 - acc: 0.9162 - val_loss: 1.2891 - val_acc: 0.6400
Epoch 416/700
1098/1098 [==============================] - 0s 304us/step - loss: 0.2774 - acc: 0.9026 - val_loss: 1.2521 - val_acc: 0.6533
Epoch 417/700
1098/1098 [==============================] - 0s 312us/step - loss: 0.2430 - acc: 0.9107 - val_loss: 1.3044 - val_acc: 0.6667
Epoch 418/700
1098/1098 [==============================] - 0s 303us/step - loss: 0.2555 - acc: 0.9153 - val_loss: 1.2456 - val_acc: 0.6800
Epoch 419/700
1098/1098 [==============================] - 0s 307us/step - loss: 0.2303 - acc: 0.9199 - val_loss: 1.4515 - val_acc: 0.6800
Epoch 420/700
1098/1098 [==============================] - 0s 304us/step - loss: 0.2771 - acc: 0.9089 - val_loss: 1.3270 - val_acc: 0.6533
Epoch 421/700
1098/1098 [================

1098/1098 [==============================] - 0s 301us/step - loss: 0.2064 - acc: 0.9262 - val_loss: 1.3327 - val_acc: 0.6533
Epoch 474/700
1098/1098 [==============================] - 0s 301us/step - loss: 0.2433 - acc: 0.9117 - val_loss: 1.4336 - val_acc: 0.6800
Epoch 475/700
1098/1098 [==============================] - 0s 296us/step - loss: 0.2245 - acc: 0.9244 - val_loss: 1.3390 - val_acc: 0.6800
Epoch 476/700
1098/1098 [==============================] - 0s 297us/step - loss: 0.2184 - acc: 0.9126 - val_loss: 1.4282 - val_acc: 0.6533
Epoch 477/700
1098/1098 [==============================] - 0s 304us/step - loss: 0.1901 - acc: 0.9317 - val_loss: 1.3663 - val_acc: 0.6667
Epoch 478/700
1098/1098 [==============================] - 0s 302us/step - loss: 0.1806 - acc: 0.9408 - val_loss: 1.4051 - val_acc: 0.6400
Epoch 479/700
1098/1098 [==============================] - 0s 303us/step - loss: 0.1977 - acc: 0.9317 - val_loss: 1.3588 - val_acc: 0.6533
Epoch 480/700
1098/1098 [================

1098/1098 [==============================] - 0s 307us/step - loss: 0.2027 - acc: 0.9281 - val_loss: 1.2616 - val_acc: 0.7067
Epoch 533/700
1098/1098 [==============================] - 0s 313us/step - loss: 0.1797 - acc: 0.9381 - val_loss: 1.4340 - val_acc: 0.7200
Epoch 534/700
1098/1098 [==============================] - 0s 312us/step - loss: 0.1781 - acc: 0.9426 - val_loss: 1.3121 - val_acc: 0.7067
Epoch 535/700
1098/1098 [==============================] - 0s 310us/step - loss: 0.1992 - acc: 0.9308 - val_loss: 1.3556 - val_acc: 0.7067
Epoch 536/700
1098/1098 [==============================] - 0s 307us/step - loss: 0.1775 - acc: 0.9381 - val_loss: 1.2724 - val_acc: 0.7333
Epoch 537/700
1098/1098 [==============================] - 0s 305us/step - loss: 0.1871 - acc: 0.9381 - val_loss: 1.1749 - val_acc: 0.6933
Epoch 538/700
1098/1098 [==============================] - 0s 309us/step - loss: 0.1839 - acc: 0.9290 - val_loss: 1.1960 - val_acc: 0.7067
Epoch 539/700
1098/1098 [================

1098/1098 [==============================] - 0s 307us/step - loss: 0.1672 - acc: 0.9472 - val_loss: 1.3904 - val_acc: 0.6933
Epoch 592/700
1098/1098 [==============================] - 0s 309us/step - loss: 0.2182 - acc: 0.9308 - val_loss: 1.2498 - val_acc: 0.6667
Epoch 593/700
1098/1098 [==============================] - 0s 316us/step - loss: 0.2268 - acc: 0.9308 - val_loss: 1.2624 - val_acc: 0.6133
Epoch 594/700
1098/1098 [==============================] - 0s 311us/step - loss: 0.2068 - acc: 0.9217 - val_loss: 1.3774 - val_acc: 0.6933
Epoch 595/700
1098/1098 [==============================] - 0s 306us/step - loss: 0.2501 - acc: 0.9062 - val_loss: 1.4159 - val_acc: 0.6933
Epoch 596/700
1098/1098 [==============================] - 0s 303us/step - loss: 0.1736 - acc: 0.9444 - val_loss: 1.4995 - val_acc: 0.7200
Epoch 597/700
1098/1098 [==============================] - 0s 306us/step - loss: 0.1923 - acc: 0.9290 - val_loss: 1.3174 - val_acc: 0.7333
Epoch 598/700
1098/1098 [================

1098/1098 [==============================] - 0s 314us/step - loss: 0.1579 - acc: 0.9399 - val_loss: 1.3696 - val_acc: 0.7067
Epoch 651/700
1098/1098 [==============================] - 0s 333us/step - loss: 0.1944 - acc: 0.9390 - val_loss: 1.5303 - val_acc: 0.6800
Epoch 652/700
1098/1098 [==============================] - 0s 311us/step - loss: 0.2342 - acc: 0.9144 - val_loss: 1.3255 - val_acc: 0.6933
Epoch 653/700
1098/1098 [==============================] - 0s 312us/step - loss: 0.2123 - acc: 0.9217 - val_loss: 1.3515 - val_acc: 0.7067
Epoch 654/700
1098/1098 [==============================] - 0s 311us/step - loss: 0.2471 - acc: 0.9126 - val_loss: 1.4001 - val_acc: 0.6800
Epoch 655/700
1098/1098 [==============================] - 0s 310us/step - loss: 0.1883 - acc: 0.9417 - val_loss: 1.3948 - val_acc: 0.6800
Epoch 656/700
1098/1098 [==============================] - 0s 314us/step - loss: 0.1711 - acc: 0.9399 - val_loss: 1.2721 - val_acc: 0.6800
Epoch 657/700
1098/1098 [================

In [145]:
# CNN D female 학습
model_C2 = create_model_CNN_D()
history = model_C2.fit(np.expand_dims(fd_train,-1),  f_c1, validation_data=(np.expand_dims(fd_test[:ns2], -1), f_c2[:ns2]), epochs=700)

Train on 1059 samples, validate on 72 samples
Epoch 1/700
1059/1059 [==============================] - 5s 5ms/step - loss: 7.8144 - acc: 0.1265 - val_loss: 2.9666 - val_acc: 0.0556
Epoch 2/700
1059/1059 [==============================] - 0s 381us/step - loss: 3.0928 - acc: 0.1416 - val_loss: 1.9396 - val_acc: 0.2361
Epoch 3/700
1059/1059 [==============================] - 0s 377us/step - loss: 2.0695 - acc: 0.1992 - val_loss: 1.8747 - val_acc: 0.2361
Epoch 4/700
1059/1059 [==============================] - 0s 388us/step - loss: 1.9599 - acc: 0.2276 - val_loss: 1.8979 - val_acc: 0.2778
Epoch 5/700
1059/1059 [==============================] - 0s 355us/step - loss: 1.9198 - acc: 0.2483 - val_loss: 1.9139 - val_acc: 0.2083
Epoch 6/700
1059/1059 [==============================] - 0s 345us/step - loss: 1.8735 - acc: 0.2606 - val_loss: 1.8520 - val_acc: 0.3333
Epoch 7/700
1059/1059 [==============================] - 0s 338us/step - loss: 1.8335 - acc: 0.2852 - val_loss: 1.7143 - val_acc: 0.36

1059/1059 [==============================] - 0s 360us/step - loss: 0.6405 - acc: 0.7677 - val_loss: 0.9793 - val_acc: 0.6389
Epoch 120/700
1059/1059 [==============================] - 0s 355us/step - loss: 0.7748 - acc: 0.7101 - val_loss: 0.9425 - val_acc: 0.6389
Epoch 121/700
1059/1059 [==============================] - 0s 359us/step - loss: 0.7113 - acc: 0.7394 - val_loss: 0.8867 - val_acc: 0.6667
Epoch 122/700
1059/1059 [==============================] - 0s 354us/step - loss: 0.6757 - acc: 0.7526 - val_loss: 0.9316 - val_acc: 0.6806
Epoch 123/700
1059/1059 [==============================] - 0s 362us/step - loss: 0.6491 - acc: 0.7517 - val_loss: 0.8684 - val_acc: 0.6528
Epoch 124/700
1059/1059 [==============================] - 0s 357us/step - loss: 0.6609 - acc: 0.7564 - val_loss: 0.9167 - val_acc: 0.6667
Epoch 125/700
1059/1059 [==============================] - 0s 352us/step - loss: 0.6597 - acc: 0.7479 - val_loss: 0.9160 - val_acc: 0.6528
Epoch 126/700
1059/1059 [================

1059/1059 [==============================] - 0s 353us/step - loss: 0.4687 - acc: 0.8300 - val_loss: 0.8835 - val_acc: 0.7222
Epoch 179/700
1059/1059 [==============================] - 0s 328us/step - loss: 0.4437 - acc: 0.8263 - val_loss: 0.8506 - val_acc: 0.6806
Epoch 180/700
1059/1059 [==============================] - 0s 348us/step - loss: 0.4672 - acc: 0.8319 - val_loss: 0.8269 - val_acc: 0.7222
Epoch 181/700
1059/1059 [==============================] - 0s 350us/step - loss: 0.4926 - acc: 0.8159 - val_loss: 0.8712 - val_acc: 0.7083
Epoch 182/700
1059/1059 [==============================] - 0s 351us/step - loss: 0.4726 - acc: 0.8168 - val_loss: 0.8971 - val_acc: 0.6528
Epoch 183/700
1059/1059 [==============================] - 0s 353us/step - loss: 0.4763 - acc: 0.8215 - val_loss: 0.8636 - val_acc: 0.6944
Epoch 184/700
1059/1059 [==============================] - 0s 349us/step - loss: 0.4910 - acc: 0.8187 - val_loss: 0.8446 - val_acc: 0.7083
Epoch 185/700
1059/1059 [================

1059/1059 [==============================] - 0s 375us/step - loss: 0.3492 - acc: 0.8763 - val_loss: 0.8961 - val_acc: 0.7222
Epoch 238/700
1059/1059 [==============================] - 0s 379us/step - loss: 0.4038 - acc: 0.8423 - val_loss: 0.8469 - val_acc: 0.6944
Epoch 239/700
1059/1059 [==============================] - 0s 377us/step - loss: 0.4170 - acc: 0.8461 - val_loss: 0.8397 - val_acc: 0.7222
Epoch 240/700
1059/1059 [==============================] - 0s 374us/step - loss: 0.3532 - acc: 0.8772 - val_loss: 0.7925 - val_acc: 0.7083
Epoch 241/700
1059/1059 [==============================] - 0s 380us/step - loss: 0.3813 - acc: 0.8612 - val_loss: 0.8665 - val_acc: 0.6806
Epoch 242/700
1059/1059 [==============================] - 0s 374us/step - loss: 0.3316 - acc: 0.8669 - val_loss: 0.8428 - val_acc: 0.6944
Epoch 243/700
1059/1059 [==============================] - 0s 377us/step - loss: 0.3717 - acc: 0.8659 - val_loss: 0.9182 - val_acc: 0.7361
Epoch 244/700
1059/1059 [================

1059/1059 [==============================] - 0s 377us/step - loss: 0.3288 - acc: 0.8867 - val_loss: 0.9861 - val_acc: 0.6667
Epoch 297/700
1059/1059 [==============================] - 0s 368us/step - loss: 0.3479 - acc: 0.8857 - val_loss: 1.0174 - val_acc: 0.6806
Epoch 298/700
1059/1059 [==============================] - 0s 374us/step - loss: 0.2965 - acc: 0.8971 - val_loss: 0.9140 - val_acc: 0.7361
Epoch 299/700
1059/1059 [==============================] - 0s 381us/step - loss: 0.3624 - acc: 0.8659 - val_loss: 0.8805 - val_acc: 0.7222
Epoch 300/700
1059/1059 [==============================] - 0s 376us/step - loss: 0.3695 - acc: 0.8678 - val_loss: 0.8560 - val_acc: 0.7500
Epoch 301/700
1059/1059 [==============================] - 0s 380us/step - loss: 0.3447 - acc: 0.8725 - val_loss: 0.9252 - val_acc: 0.6667
Epoch 302/700
1059/1059 [==============================] - 0s 375us/step - loss: 0.3632 - acc: 0.8763 - val_loss: 0.9021 - val_acc: 0.7222
Epoch 303/700
1059/1059 [================

...

1059/1059 [==============================] - 0s 351us/step - loss: 0.2169 - acc: 0.9065 - val_loss: 0.9154 - val_acc: 0.7500
Epoch 415/700
1059/1059 [==============================] - 0s 350us/step - loss: 0.2884 - acc: 0.9084 - val_loss: 0.9778 - val_acc: 0.7222
Epoch 416/700
1059/1059 [==============================] - 0s 354us/step - loss: 0.2710 - acc: 0.9150 - val_loss: 1.0332 - val_acc: 0.7222
Epoch 417/700
1059/1059 [==============================] - 0s 355us/step - loss: 0.3464 - acc: 0.8763 - val_loss: 0.9931 - val_acc: 0.7083
Epoch 418/700
1059/1059 [==============================] - 0s 348us/step - loss: 0.3745 - acc: 0.8584 - val_loss: 0.9923 - val_acc: 0.7361
Epoch 419/700
1059/1059 [==============================] - 0s 366us/step - loss: 0.2729 - acc: 0.8961 - val_loss: 0.9114 - val_acc: 0.6667
Epoch 420/700
1059/1059 [==============================] - 0s 352us/step - loss: 0.2561 - acc: 0.9008 - val_loss: 0.8389 - val_acc: 0.7361
Epoch 421/700
1059/1059 [================

1059/1059 [==============================] - 0s 351us/step - loss: 0.2613 - acc: 0.9018 - val_loss: 1.0682 - val_acc: 0.6806
Epoch 474/700
1059/1059 [==============================] - 0s 343us/step - loss: 0.2322 - acc: 0.9141 - val_loss: 0.9344 - val_acc: 0.7500
Epoch 475/700
1059/1059 [==============================] - 0s 347us/step - loss: 0.2084 - acc: 0.9292 - val_loss: 1.0022 - val_acc: 0.7222
Epoch 476/700
1059/1059 [==============================] - 0s 360us/step - loss: 0.2377 - acc: 0.9075 - val_loss: 1.0577 - val_acc: 0.7639
Epoch 477/700
1059/1059 [==============================] - 0s 347us/step - loss: 0.2305 - acc: 0.9169 - val_loss: 0.9011 - val_acc: 0.7222
Epoch 478/700
1059/1059 [==============================] - 0s 357us/step - loss: 0.2437 - acc: 0.9160 - val_loss: 0.9716 - val_acc: 0.7361
Epoch 479/700
1059/1059 [==============================] - 0s 349us/step - loss: 0.1861 - acc: 0.9254 - val_loss: 1.0240 - val_acc: 0.7222
Epoch 480/700
1059/1059 [================

1059/1059 [==============================] - 0s 337us/step - loss: 0.1882 - acc: 0.9292 - val_loss: 0.9536 - val_acc: 0.7083
Epoch 533/700
1059/1059 [==============================] - 0s 341us/step - loss: 0.2392 - acc: 0.9160 - val_loss: 0.8668 - val_acc: 0.7500
Epoch 534/700
1059/1059 [==============================] - 0s 341us/step - loss: 0.2112 - acc: 0.9235 - val_loss: 0.9590 - val_acc: 0.7222
Epoch 535/700
1059/1059 [==============================] - 0s 338us/step - loss: 0.2061 - acc: 0.9301 - val_loss: 0.9771 - val_acc: 0.6528
Epoch 536/700
1059/1059 [==============================] - 0s 343us/step - loss: 0.1778 - acc: 0.9377 - val_loss: 0.9586 - val_acc: 0.7083
Epoch 537/700
1059/1059 [==============================] - 0s 351us/step - loss: 0.2023 - acc: 0.9273 - val_loss: 0.9435 - val_acc: 0.7222
Epoch 538/700
1059/1059 [==============================] - 0s 342us/step - loss: 0.1945 - acc: 0.9311 - val_loss: 0.9993 - val_acc: 0.7361
Epoch 539/700
1059/1059 [================

1059/1059 [==============================] - 0s 326us/step - loss: 0.1584 - acc: 0.9424 - val_loss: 0.9839 - val_acc: 0.7361
Epoch 592/700
1059/1059 [==============================] - 0s 325us/step - loss: 0.2304 - acc: 0.9235 - val_loss: 1.0937 - val_acc: 0.6944
Epoch 593/700
1059/1059 [==============================] - 0s 369us/step - loss: 0.3045 - acc: 0.9018 - val_loss: 1.2631 - val_acc: 0.7222
Epoch 594/700
1059/1059 [==============================] - 0s 347us/step - loss: 0.2501 - acc: 0.9027 - val_loss: 0.9680 - val_acc: 0.7222
Epoch 595/700
1059/1059 [==============================] - 0s 350us/step - loss: 0.1678 - acc: 0.9415 - val_loss: 1.0548 - val_acc: 0.7500
Epoch 596/700
1059/1059 [==============================] - 0s 356us/step - loss: 0.1978 - acc: 0.9263 - val_loss: 1.1094 - val_acc: 0.7222
Epoch 597/700
1059/1059 [==============================] - 0s 351us/step - loss: 0.3783 - acc: 0.8820 - val_loss: 0.9383 - val_acc: 0.7778
Epoch 598/700
1059/1059 [================

Epoch 650/700
1059/1059 [==============================] - 0s 342us/step - loss: 0.1951 - acc: 0.9377 - val_loss: 0.9370 - val_acc: 0.7639
Epoch 651/700
1059/1059 [==============================] - 0s 341us/step - loss: 0.2024 - acc: 0.9367 - val_loss: 1.1300 - val_acc: 0.7222
Epoch 652/700
1059/1059 [==============================] - 0s 348us/step - loss: 0.1582 - acc: 0.9424 - val_loss: 1.0511 - val_acc: 0.7639
Epoch 653/700
1059/1059 [==============================] - 0s 357us/step - loss: 0.1615 - acc: 0.9424 - val_loss: 0.9947 - val_acc: 0.7500
Epoch 654/700
1059/1059 [==============================] - 0s 355us/step - loss: 0.1863 - acc: 0.9377 - val_loss: 1.0886 - val_acc: 0.7222
Epoch 655/700
1059/1059 [==============================] - 0s 358us/step - loss: 0.1939 - acc: 0.9339 - val_loss: 0.9278 - val_acc: 0.7500
Epoch 656/700
1059/1059 [==============================] - 0s 361us/step - loss: 0.1964 - acc: 0.9330 - val_loss: 0.9684 - val_acc: 0.7500
Epoch 657/700
1059/1059 [==

In [146]:
# CNN O male 학습
model_D = create_model_CNN_O()
history = model_D.fit(np.expand_dims(md_train,-1),  m_c1, validation_data=(np.expand_dims(md_test[:ns], -1), m_c2[:ns]), batch_size=16, epochs=400)

Train on 1098 samples, validate on 75 samples
Epoch 1/400
1098/1098 [==============================] - 5s 5ms/step - loss: 5.4618 - acc: 0.1439 - val_loss: 2.1305 - val_acc: 0.2000
Epoch 2/400
1098/1098 [==============================] - 1s 551us/step - loss: 2.1766 - acc: 0.1967 - val_loss: 2.0239 - val_acc: 0.2133
Epoch 3/400
1098/1098 [==============================] - 1s 559us/step - loss: 1.9823 - acc: 0.2459 - val_loss: 1.8745 - val_acc: 0.2800
Epoch 4/400
1098/1098 [==============================] - 1s 566us/step - loss: 1.9236 - acc: 0.2459 - val_loss: 1.8368 - val_acc: 0.2667
Epoch 5/400
1098/1098 [==============================] - 1s 559us/step - loss: 1.8127 - acc: 0.2923 - val_loss: 1.7469 - val_acc: 0.2800
Epoch 6/400
1098/1098 [==============================] - 1s 570us/step - loss: 1.6993 - acc: 0.3215 - val_loss: 1.6828 - val_acc: 0.2533
Epoch 7/400
1098/1098 [==============================] - 1s 570us/step - loss: 1.6655 - acc: 0.3670 - val_loss: 1.6177 - val_acc: 0.38

1098/1098 [==============================] - 1s 540us/step - loss: 0.3565 - acc: 0.8761 - val_loss: 1.0838 - val_acc: 0.6667
Epoch 120/400
1098/1098 [==============================] - 1s 555us/step - loss: 0.3572 - acc: 0.8816 - val_loss: 0.9394 - val_acc: 0.7067
Epoch 121/400
1098/1098 [==============================] - 1s 560us/step - loss: 0.3888 - acc: 0.8661 - val_loss: 0.9350 - val_acc: 0.6933
Epoch 122/400
1098/1098 [==============================] - 1s 539us/step - loss: 0.3791 - acc: 0.8652 - val_loss: 0.9728 - val_acc: 0.7067
Epoch 123/400
1098/1098 [==============================] - 1s 544us/step - loss: 0.3882 - acc: 0.8652 - val_loss: 1.0973 - val_acc: 0.6800
Epoch 124/400
1098/1098 [==============================] - 1s 538us/step - loss: 0.3819 - acc: 0.8506 - val_loss: 1.0777 - val_acc: 0.6667
Epoch 125/400
1098/1098 [==============================] - 1s 541us/step - loss: 0.3502 - acc: 0.8716 - val_loss: 1.0468 - val_acc: 0.6800
Epoch 126/400
1098/1098 [================

1098/1098 [==============================] - 1s 535us/step - loss: 0.3129 - acc: 0.8843 - val_loss: 1.1077 - val_acc: 0.6933
Epoch 179/400
1098/1098 [==============================] - 1s 539us/step - loss: 0.3012 - acc: 0.8989 - val_loss: 1.1084 - val_acc: 0.7600
Epoch 180/400
1098/1098 [==============================] - 1s 507us/step - loss: 0.3038 - acc: 0.8880 - val_loss: 1.1018 - val_acc: 0.6933
Epoch 181/400
1098/1098 [==============================] - 1s 505us/step - loss: 0.3018 - acc: 0.9053 - val_loss: 1.1624 - val_acc: 0.7333
Epoch 182/400
1098/1098 [==============================] - 1s 500us/step - loss: 0.2639 - acc: 0.9135 - val_loss: 1.1389 - val_acc: 0.7333
Epoch 183/400
1098/1098 [==============================] - 1s 503us/step - loss: 0.2833 - acc: 0.8953 - val_loss: 1.2173 - val_acc: 0.7467
Epoch 184/400
1098/1098 [==============================] - 1s 503us/step - loss: 0.2910 - acc: 0.8953 - val_loss: 1.1411 - val_acc: 0.7600
Epoch 185/400
1098/1098 [================

1098/1098 [==============================] - 1s 530us/step - loss: 0.2725 - acc: 0.8925 - val_loss: 1.1636 - val_acc: 0.6800
Epoch 238/400
1098/1098 [==============================] - 1s 514us/step - loss: 0.2820 - acc: 0.9053 - val_loss: 1.3523 - val_acc: 0.6667
Epoch 239/400
1098/1098 [==============================] - 1s 530us/step - loss: 0.2644 - acc: 0.8989 - val_loss: 1.2448 - val_acc: 0.6933
Epoch 240/400
1098/1098 [==============================] - 1s 516us/step - loss: 0.2595 - acc: 0.9098 - val_loss: 1.3091 - val_acc: 0.7333
Epoch 241/400
1098/1098 [==============================] - 1s 544us/step - loss: 0.2545 - acc: 0.9044 - val_loss: 1.2044 - val_acc: 0.7467
Epoch 242/400
1098/1098 [==============================] - 1s 520us/step - loss: 0.2364 - acc: 0.9180 - val_loss: 1.2164 - val_acc: 0.7067
Epoch 243/400
1098/1098 [==============================] - 1s 527us/step - loss: 0.2277 - acc: 0.9189 - val_loss: 1.3333 - val_acc: 0.6933
Epoch 244/400
1098/1098 [================

1098/1098 [==============================] - 1s 531us/step - loss: 0.2223 - acc: 0.9290 - val_loss: 1.3031 - val_acc: 0.7333
Epoch 297/400
1098/1098 [==============================] - 1s 512us/step - loss: 0.2067 - acc: 0.9253 - val_loss: 1.4069 - val_acc: 0.7200
Epoch 298/400
1098/1098 [==============================] - 1s 518us/step - loss: 0.2166 - acc: 0.9235 - val_loss: 1.4020 - val_acc: 0.7333
Epoch 299/400
1098/1098 [==============================] - 1s 515us/step - loss: 0.2210 - acc: 0.9208 - val_loss: 1.3289 - val_acc: 0.7200
Epoch 300/400
1098/1098 [==============================] - 1s 527us/step - loss: 0.2130 - acc: 0.9290 - val_loss: 1.3413 - val_acc: 0.7067
Epoch 301/400
1098/1098 [==============================] - 1s 534us/step - loss: 0.2530 - acc: 0.9117 - val_loss: 1.3679 - val_acc: 0.7467
Epoch 302/400
1098/1098 [==============================] - 1s 513us/step - loss: 0.2102 - acc: 0.9281 - val_loss: 1.3485 - val_acc: 0.7067
Epoch 303/400
1098/1098 [================

1098/1098 [==============================] - 1s 525us/step - loss: 0.1780 - acc: 0.9435 - val_loss: 1.2664 - val_acc: 0.6933
Epoch 356/400
1098/1098 [==============================] - 1s 537us/step - loss: 0.2124 - acc: 0.9253 - val_loss: 1.3194 - val_acc: 0.7200
Epoch 357/400
1098/1098 [==============================] - 1s 518us/step - loss: 0.2197 - acc: 0.9199 - val_loss: 1.3034 - val_acc: 0.7467
Epoch 358/400
1098/1098 [==============================] - 1s 517us/step - loss: 0.1881 - acc: 0.9326 - val_loss: 1.2251 - val_acc: 0.7067
Epoch 359/400
1098/1098 [==============================] - 1s 520us/step - loss: 0.1819 - acc: 0.9381 - val_loss: 1.2807 - val_acc: 0.7200
Epoch 360/400
1098/1098 [==============================] - 1s 513us/step - loss: 0.1659 - acc: 0.9481 - val_loss: 1.2419 - val_acc: 0.7333
Epoch 361/400
1098/1098 [==============================] - 1s 513us/step - loss: 0.1593 - acc: 0.9399 - val_loss: 1.3432 - val_acc: 0.6933
Epoch 362/400
1098/1098 [================

In [147]:
# CNN O female 학습
model_D2 = create_model_CNN_O()
history = model_D2.fit(np.expand_dims(fd_train,-1),  f_c1, validation_data=(np.expand_dims(fd_test[:ns2], -1), f_c2[:ns2]), batch_size=16, epochs=400)

Train on 1059 samples, validate on 72 samples
Epoch 1/400
1059/1059 [==============================] - 5s 5ms/step - loss: 5.2888 - acc: 0.1662 - val_loss: 2.0737 - val_acc: 0.1528
Epoch 2/400
1059/1059 [==============================] - 1s 549us/step - loss: 2.1619 - acc: 0.1747 - val_loss: 1.8891 - val_acc: 0.2222
Epoch 3/400
1059/1059 [==============================] - 1s 566us/step - loss: 1.9836 - acc: 0.2219 - val_loss: 1.8449 - val_acc: 0.2500
Epoch 4/400
1059/1059 [==============================] - 1s 566us/step - loss: 1.9807 - acc: 0.2200 - val_loss: 1.8617 - val_acc: 0.2083
Epoch 5/400
1059/1059 [==============================] - 1s 586us/step - loss: 1.9256 - acc: 0.2276 - val_loss: 1.8470 - val_acc: 0.2778
Epoch 6/400
1059/1059 [==============================] - 1s 565us/step - loss: 1.8848 - acc: 0.2493 - val_loss: 1.7388 - val_acc: 0.3472
Epoch 7/400
1059/1059 [==============================] - 1s 556us/step - loss: 1.8341 - acc: 0.2786 - val_loss: 1.7971 - val_acc: 0.29

1059/1059 [==============================] - 1s 567us/step - loss: 0.5294 - acc: 0.7941 - val_loss: 0.9712 - val_acc: 0.6944
Epoch 120/400
1059/1059 [==============================] - 1s 566us/step - loss: 0.6001 - acc: 0.7686 - val_loss: 0.9683 - val_acc: 0.6528
Epoch 121/400
1059/1059 [==============================] - 1s 557us/step - loss: 0.5460 - acc: 0.8017 - val_loss: 1.0031 - val_acc: 0.6528
Epoch 122/400
1059/1059 [==============================] - 1s 573us/step - loss: 0.5207 - acc: 0.8017 - val_loss: 0.9518 - val_acc: 0.6806
Epoch 123/400
1059/1059 [==============================] - 1s 575us/step - loss: 0.5134 - acc: 0.8178 - val_loss: 1.0277 - val_acc: 0.6806
Epoch 124/400
1059/1059 [==============================] - 1s 563us/step - loss: 0.5466 - acc: 0.7941 - val_loss: 0.9696 - val_acc: 0.7083
Epoch 125/400
1059/1059 [==============================] - 1s 586us/step - loss: 0.5260 - acc: 0.7941 - val_loss: 0.9507 - val_acc: 0.6667
Epoch 126/400
1059/1059 [================

1059/1059 [==============================] - 1s 560us/step - loss: 0.3835 - acc: 0.8517 - val_loss: 1.1488 - val_acc: 0.6806
Epoch 179/400
1059/1059 [==============================] - 1s 560us/step - loss: 0.3589 - acc: 0.8602 - val_loss: 1.0211 - val_acc: 0.6806
Epoch 180/400
1059/1059 [==============================] - 1s 567us/step - loss: 0.3628 - acc: 0.8640 - val_loss: 1.1191 - val_acc: 0.6667
Epoch 181/400
1059/1059 [==============================] - 1s 557us/step - loss: 0.4258 - acc: 0.8432 - val_loss: 1.1117 - val_acc: 0.6250
Epoch 182/400
1059/1059 [==============================] - 1s 577us/step - loss: 0.3751 - acc: 0.8697 - val_loss: 1.1744 - val_acc: 0.6528
Epoch 183/400
1059/1059 [==============================] - 1s 559us/step - loss: 0.3692 - acc: 0.8612 - val_loss: 1.1192 - val_acc: 0.6944
Epoch 184/400
1059/1059 [==============================] - 1s 574us/step - loss: 0.3676 - acc: 0.8621 - val_loss: 1.1516 - val_acc: 0.6528
Epoch 185/400
1059/1059 [================

1059/1059 [==============================] - 1s 531us/step - loss: 0.3340 - acc: 0.8810 - val_loss: 1.0478 - val_acc: 0.6806
Epoch 238/400
1059/1059 [==============================] - 1s 526us/step - loss: 0.3142 - acc: 0.8895 - val_loss: 1.1763 - val_acc: 0.6944
Epoch 239/400
1059/1059 [==============================] - 1s 517us/step - loss: 0.2995 - acc: 0.8820 - val_loss: 1.0953 - val_acc: 0.6806
Epoch 240/400
1059/1059 [==============================] - 1s 526us/step - loss: 0.3010 - acc: 0.9046 - val_loss: 1.1135 - val_acc: 0.6944
Epoch 241/400
1059/1059 [==============================] - 1s 520us/step - loss: 0.2873 - acc: 0.9056 - val_loss: 1.1940 - val_acc: 0.6944
Epoch 242/400
1059/1059 [==============================] - ETA: 0s - loss: 0.2985 - acc: 0.894 - 1s 555us/step - loss: 0.3002 - acc: 0.8933 - val_loss: 1.1776 - val_acc: 0.6944
Epoch 243/400
1059/1059 [==============================] - 1s 527us/step - loss: 0.2945 - acc: 0.8942 - val_loss: 1.2954 - val_acc: 0.6528
Epo

Epoch 296/400
1059/1059 [==============================] - 1s 526us/step - loss: 0.2501 - acc: 0.9093 - val_loss: 1.3375 - val_acc: 0.6667
Epoch 297/400
1059/1059 [==============================] - 1s 531us/step - loss: 0.2814 - acc: 0.8942 - val_loss: 1.4226 - val_acc: 0.6111
Epoch 298/400
1059/1059 [==============================] - 1s 529us/step - loss: 0.2773 - acc: 0.8999 - val_loss: 1.2162 - val_acc: 0.6528
Epoch 299/400
1059/1059 [==============================] - 1s 526us/step - loss: 0.2622 - acc: 0.9093 - val_loss: 1.3939 - val_acc: 0.6667
Epoch 300/400
1059/1059 [==============================] - 1s 531us/step - loss: 0.2477 - acc: 0.9084 - val_loss: 1.4806 - val_acc: 0.6389
Epoch 301/400
1059/1059 [==============================] - 1s 530us/step - loss: 0.2758 - acc: 0.9093 - val_loss: 1.4798 - val_acc: 0.6389
Epoch 302/400
1059/1059 [==============================] - 1s 533us/step - loss: 0.2669 - acc: 0.9018 - val_loss: 1.4835 - val_acc: 0.6389
Epoch 303/400
1059/1059 [==

In [148]:
# SVM 학습
clf = svm.SVC(kernel='linear')
clf2 = svm.SVC(kernel='linear')
clf.fit(md_train, ml_train)
clf2.fit(fd_train, fl_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [162]:
# 모델 별 정확도 체크

print("LSTM score-male: ",model_A.evaluate(np.expand_dims(md_test[ns:], -1),  m_c2[ns:]))
print("LSTM score-female: ",model_A2.evaluate(np.expand_dims(fd_test[ns2:], -1),  f_c2[ns2:]))

print("CNN_Score-male : ",model_B.evaluate(np.expand_dims(md_test[ns:], -1),  m_c2[ns:]))
print("CNN_Score-female : ",model_B2.evaluate(np.expand_dims(fd_test[ns2:], -1),  f_c2[ns2:]))

print("CNN_DropOut_Score-male : ",model_C.evaluate(np.expand_dims(md_test[ns:], -1),  m_c2[ns:]))
print("CNN_DropOut_Score-female : ",model_C2.evaluate(np.expand_dims(fd_test[ns2:], -1),  f_c2[ns2:]))

print("CNN_Opt_Score-male : ",model_D.evaluate(np.expand_dims(md_test[ns:], -1),  m_c2[ns:]))
print("CNN_Opt_Score-female : ",model_D2.evaluate(np.expand_dims(fd_test[ns2:], -1),  f_c2[ns2:]))

print("svm_Score-male : ", clf.score(md_test,ml_test))
print("svm_Score-female : ", clf2.score(fd_test,fl_test))




75/75 [==============================] - 1s 8ms/step
LSTM score-male:  [2.9610189326604206, 0.6533333373069763]
73/73 [==============================] - 1s 8ms/step
LSTM score-female:  [2.217450497901603, 0.6986301394358073]
75/75 [==============================] - 0s 718us/step
CNN_Score-male :  [0.7876862247784933, 0.8399999992052714]
73/73 [==============================] - 0s 833us/step
CNN_Score-female :  [0.97192850096585, 0.7671232884877348]
75/75 [==============================] - 0s 625us/step
CNN_DropOut_Score-male :  [1.3038557084401448, 0.7200000031789144]
73/73 [==============================] - 0s 792us/step
CNN_DropOut_Score-female :  [1.0279900537778253, 0.7534246591672505]
75/75 [==============================] - 0s 612us/step
CNN_Opt_Score-male :  [1.2957885519663492, 0.6666666650772095]
73/73 [==============================] - 0s 628us/step
CNN_Opt_Score-female :  [1.5981806802423033, 0.6986301378028034]
svm_Score-male :  0.6533333333333333
svm_Score-female :  0.6896

In [163]:
# bipolar 데이터 MFCC 변환_남성

root_dir = "./bipolar_audio/male/" 

    
male_bipolar_labels = []
male_bipolar_data = []

for file_name in os.listdir(root_dir):
    
    male_bipolar_labels.append(file_name)
    wav_file_name = os.path.join(root_dir, file_name)
    male_bipolar_data.append(extract_mfcc(wav_file_name))


In [164]:
# bipolar 데이터 MFCC 변환_여성

root_dir = "./bipolar_audio/female/" 

    
female_bipolar_labels = []
female_bipolar_data = []

for file_name in os.listdir(root_dir):
    
    female_bipolar_labels.append(file_name)
    wav_file_name = os.path.join(root_dir, file_name)
    female_bipolar_data.append(extract_mfcc(wav_file_name))


In [170]:
# 데이터, 라벨 입력

m_b_data = np.asarray(male_bipolar_data)
m_b_labels = male_bipolar_labels

f_b_data = np.asarray(female_bipolar_data)
f_b_labels = female_bipolar_labels

In [171]:
# 감정 분석 모델에 bipolar 데이터 입력

A_pred = model_A.predict_classes(np.expand_dims(m_b_data, -1))
B_pred = model_B.predict_classes(np.expand_dims(m_b_data, -1))
C_pred = model_C.predict_classes(np.expand_dims(m_b_data, -1))
D_pred = model_D.predict_classes(np.expand_dims(m_b_data, -1))
E_pred = clf.predict(m_b_data)

A2_pred = model_A2.predict_classes(np.expand_dims(f_b_data, -1))
B2_pred = model_B2.predict_classes(np.expand_dims(f_b_data, -1))
C2_pred = model_C2.predict_classes(np.expand_dims(f_b_data, -1))
D2_pred = model_D2.predict_classes(np.expand_dims(f_b_data, -1))
E2_pred = clf2.predict(f_b_data)

In [172]:
# 감정 분류 결과 출력

emotions = ["neutral", "calm", "happy", "sad","angry","fearful", "disgust", "surprised"]
    
data = {}
data['LSTM'] =[]
data['CNN'] =[]
data['CNN_D']=[]
data['CNN_O']=[]
data['SVM'] =[]

mf_labels = m_b_labels+ f_b_labels

for i in range(0,2):    
    data['LSTM'].append(emotions[A_pred[i]])
    data['CNN'].append(emotions[B_pred[i]])
    data['CNN_D'].append(emotions[C_pred[i]])
    data['CNN_O'].append(emotions[D_pred[i]])
    data['SVM'].append(emotions[E_pred[i]])
    
for i in range(0,8):    
    data['LSTM'].append(emotions[A2_pred[i]])
    data['CNN'].append(emotions[B2_pred[i]])
    data['CNN_D'].append(emotions[C2_pred[i]])
    data['CNN_O'].append(emotions[D2_pred[i]])
    data['SVM'].append(emotions[E2_pred[i]])
    
    
frame = pd.DataFrame(data, index=mf_labels)
frame

,LSTM,CNN,CNN_D,CNN_O,SVM
01-neutral-01.wav,happy,disgust,angry,sad,calm
01-neutral-02.wav,calm,disgust,happy,disgust,calm
02-depressed-01.wav,calm,calm,calm,disgust,disgust
02-depressed-02.wav,disgust,calm,calm,disgust,angry
02-mania-01-02.wav,disgust,happy,disgust,disgust,angry
02-mania-01.wav,disgust,angry,happy,happy,angry
02-mania-02-02.wav,disgust,angry,happy,disgust,happy
02-mania-02.wav,disgust,angry,angry,disgust,angry
02-neutral-01.wav,disgust,calm,calm,disgust,disgust
02-neutral-02.wav,disgust,disgust,disgust,disgust,angry
